In [ ]:
## https://ayoolafelix.hashnode.dev/how-to-permanently-install-a-module-on-google-colab-ckixqrvs40su044s187y274tc
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [40]:
#Importing Libraries
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime, timedelta
from pytz import timezone
import math
import yfinance as yf
from datetime import date
from datetime import datetime

##Backtesting Strategy: Momentum Strategy >> ATR Based Trailing SL in NIFTY FUT

In [41]:
##https://medium.datadriveninvestor.com/the-supertrend-implementing-screening-backtesting-in-python-70e8f88f383d

def Supertrend(data,atr_period, multiplier):
    
    high = data['High']
    low = data['Low']
    close = data['Close']


    # calculate ATRF
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # print(atr)
    # data['atr'] = data['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(data)
    
    for i in range(1, len(data.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'final_lowerband': final_lowerband,
        'final_upperband': final_upperband
    }, index=data.index)
        
n = 2500 #No of days
atr_period = 21
atr_multiplier = 3
symbol = '^NSEI' #^NSEI ,USDINR=X,^NSEBANK
lot_size = 50
today_yf = datetime.now()
today_yf = today_yf.strftime("%Y-%m-%d")   #Different API used,different datetime formats
last_one_year_yf = datetime.today() - timedelta(days= n)
last_one_year_yf = last_one_year_yf.strftime("%Y-%m-%d")
data= yf.download( symbol, start = last_one_year_yf, interval="1d" ) #period=f"{n}d"

"""NSE PY"""
# today = datetime.now()  #Today Datetime
# one_year = datetime.today() - timedelta(days=n) #Subrtracting todays date with 370 Days
# nf_vix = get_history(symbol="INDIAVIX",start=one_year,end=today,index=True)
# print("NSEPY INDIA VIX Dataframe",nf_vix.head())

today = datetime.now()
start = datetime.today() - timedelta(days= n)
start = start.strftime("%Y,%m,%d")
start = datetime.strptime(start, "%Y,%m,%d") #https://stackoverflow.com/questions/466345/convert-string-jun-1-2005-133pm-into-datetime
end = today.strftime("%Y,%m,%d")   
end = datetime.strptime(end,"%Y,%m,%d")

nf_vix = get_history(symbol="INDIAVIX", start= start, end= end, index=True)
data = data.reset_index()
nf_vix = nf_vix.reset_index()
data['vix'] = nf_vix['Close'].round(2)
data.set_index('Date', inplace=True)

# data= data.between_time('9:00','16:00') #https://stackoverflow.com/questions/19179214/selecting-data-between-specific-hours-in-a-pandas-dataframe

supertrend = Supertrend(data,atr_period, atr_multiplier)
data= data.join(supertrend)
# data.loc[(data['atrts'] > df['Close']),'signal_sell']= df["atrts"] #"sell"
# data.loc[(data['atrts'] < df['Close']),'signal_buy']= df["atrts"]  #"buy"
# data['downtrend'] = ~data['Supertrend']
data.head()

def backtest_supertrend_buy(data):
    is_uptrend = data['Supertrend']
    close = data['Close']
    vix = data['vix']
    
    # initial condition
    in_position = False
    entry = []
    exit = []
    
    for i in range(2, len(data)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            entry.append((data.index[i].strftime("%Y/%m/%d"), close[i], vix[i])) #%H:%M:%S
            in_position = True
            # print(f'Buy {share} shares at {round(close[i],2)} on {data.index[i].strftime("%Y/%m/%d %H:%M:%S")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            exit.append((data.index[i].strftime("%Y/%m/%d"), close[i], vix[i])) #%H:%M:%S
            in_position = False

    return entry, exit
entry, exit = backtest_supertrend_buy(data)
df2 = (pd.DataFrame(entry, columns=['Buy Date', 'ln_entry','ln_vix_buy']).explode('ln_entry', ignore_index=True))
df3 = (pd.DataFrame(exit, columns=['Buy Exit Date', 'ln_exit','ln_vix_sell']).explode('ln_exit', ignore_index=True))
df4 = pd.concat([df2, df3], axis=1)
df4['position'] = "long buy"
df4['symbol'] = symbol
df4['diff_buy'] = round(df4['ln_exit'] - df4['ln_entry'])
df4['lot_size'] = lot_size
df4['ln_pl'] = df4['diff_buy'] * lot_size #50 is lot size for NF Futures
sum_ln = df4['ln_pl'].sum()
print(f"Total Profit/Loss from long strategy for {symbol} in ₹ per lot",sum_ln)
ln_max_loss = df4['ln_pl'].min()
print(f"Max Loss in Long Position in Supertrend Strategy for {symbol} per lot in ₹",ln_max_loss) 
ln_max_profit = df4['ln_pl'].max()
print(f"Max Profit in Long Position in Supertrend Strategy for {symbol} per lot in ₹",ln_max_profit) 
ln_loss = (df4['ln_pl'][df4['ln_pl']<0].count())
ln_profit = (df4['ln_pl'][df4['ln_pl']>0].count())

ln_profit = (df4['ln_pl'][df4['ln_pl']>0].count())
print("Number of Trades in Profit in Long Buy:",ln_profit)
ln_loss = (df4['ln_pl'][df4['ln_pl']<0].count())
print("Number of Trades in Loss in Long Buy:",ln_loss)

print("Total Trades:", df4.shape[0])
ln_loss_pct = round((ln_loss/df4.shape[0])*100)
ln_profit_pct = round((ln_profit/df4.shape[0])*100)
print(f"Count of Trade percentage in loss for {symbol} in percent (POL)",ln_loss_pct,"%")
print(f"Count of Trade percentage in Profit for {symbol} in percent (POP)",ln_profit_pct,"%")
ln_pl_ratio = round(ln_profit / ln_loss,2)
print("p&l ratio for Long Buy Strategy", ln_pl_ratio) 
ln_avg_profit = round((df4['ln_pl'][df4['ln_pl']>0].mean()))
ln_avg_loss = round((df4['ln_pl'][df4['ln_pl']<0].mean()))
print("Avg Profit in Long Buy in ₹:",ln_avg_profit)
print("Avg Loss in Long Buy in ₹:",ln_avg_loss)

df4
# max_drawdown = (ln_max_profit - (ln_max_profit - ln_max_loss)) / (ln_max_profit - ln_max_loss)
# max_drawdown|

[*********************100%***********************]  1 of 1 completed
Total Profit/Loss from long strategy for ^NSEI in ₹ per lot 630000.0
Max Loss in Long Position in Supertrend Strategy for ^NSEI per lot in ₹ -30450.0
Max Profit in Long Position in Supertrend Strategy for ^NSEI per lot in ₹ 137450.0
Number of Trades in Profit in Long Buy: 15
Number of Trades in Loss in Long Buy: 7
Total Trades: 22
Count of Trade percentage in loss for ^NSEI in percent (POL) 32 %
Count of Trade percentage in Profit for ^NSEI in percent (POP) 68 %
p&l ratio for Long Buy Strategy 2.14
Avg Profit in Long Buy in ₹: 47593
Avg Loss in Long Buy in ₹: -11986


,Buy Date,ln_entry,ln_vix_buy,Buy Exit Date,ln_exit,ln_vix_sell,position,symbol,diff_buy,lot_size,ln_pl
0,2016/03/10,7486.149902,17.50,2016/09/29,8591.250000,14.56,long buy,^NSEI,1105.0,50,55250.0
1,2017/01/05,8273.799805,15.28,2017/08/10,9820.250000,11.94,long buy,^NSEI,1546.0,50,77300.0
2,2017/09/01,9974.400391,14.10,2017/09/25,9872.599609,11.47,long buy,^NSEI,-102.0,50,-5100.0
3,2017/10/10,10016.950195,13.17,2017/11/14,10186.599609,11.91,long buy,^NSEI,170.0,50,8500.0
4,2017/11/27,10399.549805,13.46,2017/12/01,10121.799805,14.04,long buy,^NSEI,-278.0,50,-13900.0
5,2017/12/15,10333.250000,15.09,2018/02/02,10760.599609,13.98,long buy,^NSEI,427.0,50,21350.0
6,2018/04/09,10379.349609,15.54,2018/05/21,10516.700195,13.76,long buy,^NSEI,137.0,50,6850.0
7,2018/06/07,10768.349609,13.14,2018/09/05,11476.950195,12.90,long buy,^NSEI,709.0,50,35450.0
8,2018/11/07,10598.400391,19.04,2019/02/18,10640.950195,18.43,long buy,^NSEI,43.0,50,2150.0
9,2019/03/06,11053.000000,15.73,2019/05/08,11359.450195,20.96,long buy,^NSEI,306.0,50,15300.0


In [42]:
def backtest_supertrend_sell(data):
    # is_downtrend = data['downtrend']
    is_uptrend = data['Supertrend']
    close = data['Close']
    vix = data['vix']
    
    # initial condition
    in_position = False
    entry_sell = []
    exit_sell = []
    
    for i in range(2, len(data)):
        # if not in position & price is on uptrend -> buy
        if not in_position and not is_uptrend[i]:
            # share = math.floor(equity / close[i] / 100) * 100
            # equity -= share * close[i]
            entry_sell.append((data.index[i].strftime("%Y/%m/%d"), close[i], vix[i])) # %H:%M:%S
            in_position = True
            # print(f'Sell {share} shares at {round(close[i],2)} on {data.index[i].strftime("%Y/%m/%d %H:%M:%S")}')
        elif in_position and is_uptrend[i]:
            # equity += share * close[i]
            exit_sell.append((data.index[i].strftime("%Y/%m/%d"), close[i], vix[i])) # %H:%M:%S
            in_position = False
            # print(f'Buy at {round(close[i],2)} on {data.index[i].strftime("%Y/%m/%d %H:%M:%S")}')
    return entry_sell, exit_sell
entry_sell, exit_sell= backtest_supertrend_sell(data)
df_sell_1 = (pd.DataFrame(entry_sell, columns=['Sell Date', 'sh_entry','sh_vix_entry']).explode('sh_entry', ignore_index=True))
df_sell_2 = (pd.DataFrame(exit_sell, columns=['Sell Exit Date', 'sh_exit','sh_vix_exit']).explode('sh_exit', ignore_index=True))
df_sell = pd.concat([df_sell_1, df_sell_2], axis=1)
df_sell['diff_sell'] = df_sell['sh_entry'] - df_sell['sh_exit']
df_sell['lot_size'] = lot_size
df_sell['sh_pl'] = round(df_sell['diff_sell'] * lot_size)
df_sell['position'] = "short sell"
df_sell['symbol'] = symbol
sum_sh = df_sell['sh_pl'].sum()
print(f"Total Profit/Loss from short strategy for {symbol} in ₹ per lot",sum_sh)
sh_max_loss = df_sell['sh_pl'].min()
print(f"Max Loss in short Position in Supertrend Strategy for {symbol} per lot in ₹:",sh_max_loss) 
sh_max_profit = df_sell['sh_pl'].max()
print(f"Max Profit in short Position in Supertrend Strategy for {symbol} per lot in ₹:",sh_max_profit) 

sh_loss = (df_sell['sh_pl'][df_sell['sh_pl']<0].count())
print("Number of Trades in Loss in Short Sell:",sh_loss)
sh_profit = (df_sell['sh_pl'][df_sell['sh_pl']>0].count())
print("Number of Trades in Profit in Short Sell:",sh_profit)
total_trade_sh = df_sell.shape[0] - 1
print("Total Trades in Short Sell Strategy:", total_trade_sh)
# sh_count_loss_trades = (df_sell['sh_pl'] < 0).sum()
# print(sh_count_loss_trades)
# sh_count_profit_trades = (df_sell['sh_pl'] > 0).sum()
# print(sh_count_profit_trades)

# sh_trade_count = df_sell.shape[0]l
sh_loss_pct = round((sh_loss/total_trade_sh)*100)
sh_profit_pct = round((sh_profit/total_trade_sh)*100)
print(f"Count of Trade percentage in Loss for {symbol} in percent (POL)",sh_loss_pct,"%")
print(f"Count of Trade percentage in Profit for {symbol} in percent (POP)",sh_profit_pct,"%")
sh_pl_ratio = round(sh_profit / sh_loss,2)
print("P&L Ration:", sh_pl_ratio)   #Ideally should be greater than 1 for better Risk Reward Ratio
sh_avg_profit = round((df_sell['sh_pl'][df_sell['sh_pl']>0].mean()))
sh_avg_loss = round((df_sell['sh_pl'][df_sell['sh_pl']<0].mean()))
print("Avg Profit in Short Sell in ₹:",sh_avg_profit)
print("Avg Loss in Short Sell in ₹:",sh_avg_loss)
# df_sell

Total Profit/Loss from short strategy for ^NSEI in ₹ per lot 90945.0
Max Loss in short Position in Supertrend Strategy for ^NSEI per lot in ₹: -34017.0
Max Profit in short Position in Supertrend Strategy for ^NSEI per lot in ₹: 134300.0
Number of Trades in Loss in Short Sell: 13
Number of Trades in Profit in Short Sell: 8
Total Trades in Short Sell Strategy: 21
Count of Trade percentage in Loss for ^NSEI in percent (POL) 62 %
Count of Trade percentage in Profit for ^NSEI in percent (POP) 38 %
P&L Ration: 0.62
Avg Profit in Short Sell in ₹: 39692
Avg Loss in Short Sell in ₹: -17430


In [43]:
df_sell

,Sell Date,sh_entry,sh_vix_entry,Sell Exit Date,sh_exit,sh_vix_exit,diff_sell,lot_size,sh_pl,position,symbol
0,2016/09/29,8591.250000,14.56,2017/01/05,8273.799805,15.28,317.450195,50,15873.0,short sell,^NSEI
1,2017/08/10,9820.250000,11.94,2017/09/01,9974.400391,14.10,-154.150391,50,-7708.0,short sell,^NSEI
2,2017/09/25,9872.599609,11.47,2017/10/10,10016.950195,13.17,-144.350586,50,-7218.0,short sell,^NSEI
3,2017/11/14,10186.599609,11.91,2017/11/27,10399.549805,13.46,-212.950195,50,-10648.0,short sell,^NSEI
4,2017/12/01,10121.799805,14.04,2017/12/15,10333.250000,15.09,-211.450195,50,-10573.0,short sell,^NSEI
5,2018/02/02,10760.599609,13.98,2018/04/09,10379.349609,15.54,381.250000,50,19062.0,short sell,^NSEI
6,2018/05/21,10516.700195,13.76,2018/06/07,10768.349609,13.14,-251.649414,50,-12582.0,short sell,^NSEI
7,2018/09/05,11476.950195,12.90,2018/11/07,10598.400391,19.04,878.549805,50,43927.0,short sell,^NSEI
8,2019/02/18,10640.950195,18.43,2019/03/06,11053.000000,15.73,-412.049805,50,-20602.0,short sell,^NSEI
9,2019/05/08,11359.450195,20.96,2019/05/20,11828.250000,23.23,-468.799805,50,-23440.0,short sell,^NSEI


In [44]:
# df_sup = pd.concat([df4,df_sell])
# df_sup

df_sup_ln = df4.copy()
df_sup_ln = df_sup_ln.rename({'Buy Date': 'entry_date', 'Buy Exit Date': 'exit_date','ln_entry': 'entry','ln_exit': 'exit','ln_vix_buy': 'vix_entry','ln_vix_sell': 'vix_exit',
                              'diff_buy': 'diff','ln_pl': 'pl'}, axis=1)
df_sup_sh = df_sell.copy()
df_sup_sh = df_sup_sh.rename({'Sell Date': 'entry_date', 'Sell Exit Date': 'exit_date','sh_entry': 'entry','sh_exit': 'exit', 'sh_vix_entry': 'vix_entry','sh_vix_exit': 'vix_exit',
                              'diff_sell': 'diff','sh_pl': 'pl'}, axis=1)
df_sup = pd.concat([df_sup_ln,df_sup_sh])

##***********************************************************************
# df_sup[['entry_date','exit_date']] = df_sup[['entry_date','exit_date']].apply(pd.to_datetime, format='%Y/%m/%d %H:%M:%S.%f') #.%f
# Remove spaces. Have in mind this will remove all spaces.
# df_sup.iloc[:, 0:2] = df_sup.iloc[:, 0:2].apply(pd.to_datetime, errors='coerce')
df_sup = df_sup.sort_values(['entry_date'] ) #, ascending=[0]
df_sup[['entry_date','exit_date']] = df_sup[['entry_date','exit_date']].apply(pd.to_datetime) #if conversion required
df_sup['days_in_trade'] = (df_sup['exit_date'] - df_sup['entry_date']).dt.days
streak = df_sup['pl'].to_list()

##https://www.geeksforgeeks.org/python-maximum-length-consecutive-positive-elements/
from collections import defaultdict
from itertools import groupby
counter = defaultdict(list)
for key, val in groupby(streak, lambda ele: "plus" if ele >= 0 else "minus"):
    counter[key].append(len(list(val)))
res = []
for key in ('plus', 'minus'):
    res.append(counter[key])
df_sup = df_sup.round()
print("Overall Winning Streak " + str(max(res[0])))
print("Overall Losing Streak : " + str(max(res[1])))
max_profit = df_sup['pl'].max()
max_loss = df_sup['pl'].min()
total_pl = df_sup['pl'].sum()
print(f"Max Profit Overall in Supertrend Strategy for {symbol} per lot in ₹",max_profit) 
print(f"Max Loss Overall in Supertrend Strategy for {symbol} per lot in ₹",max_loss)
print(f"Total Overall P&L in Supertrend Strategy for {symbol} per lot in ₹",total_pl)
m = df_sup['pl'].gt(0).map({True: 'positive', False: 'negative'})
streak_sum = df_sup.groupby([m, m.ne(m.shift()).cumsum()])['pl'].agg(['count', 'sum'])
sum_out = streak_sum.loc[streak_sum.groupby(level=0)['count'].idxmax(), 'sum'].droplevel(1)
print(f"Overall Losing Streak Loss in Supertrend Strategy for {symbol} FUT per lot in ₹",sum_out.values[0])
print(f"Overall Winning Streak Profit in Supertrend Strategy for {symbol} FUT per lot in ₹",sum_out.values[1])
total_trades = df_sup.shape[0] - 1
print("Total Trades:",total_trades)
loss_making_trade = (df_sup['pl'] < 0).sum()
print("Count of Trades having Loss:", loss_making_trade)
profitable_trade = (df_sup['pl'] > 0).sum()
print("Count of Trades having Profit:", profitable_trade)
df_sup_pl_ratio = round(profitable_trade/loss_making_trade,2)
print("SuperTrend Strategy P&L ratio", df_sup_pl_ratio)
df_sup_loss_pct = round((loss_making_trade/total_trades)*100)
df_sup_profit_pct = round((profitable_trade/total_trades)*100)
print(f"Count of Trade percentage in Loss for {symbol} in percent (POL)",df_sup_loss_pct,"%")
print(f"Count of Trade percentage in Profit for {symbol} in percent (POP)",df_sup_profit_pct,"%")

Overall Winning Streak 4
Overall Losing Streak : 3
Max Profit Overall in Supertrend Strategy for ^NSEI per lot in ₹ 137450.0
Max Loss Overall in Supertrend Strategy for ^NSEI per lot in ₹ -34017.0
Total Overall P&L in Supertrend Strategy for ^NSEI per lot in ₹ 720945.0
Overall Losing Streak Loss in Supertrend Strategy for ^NSEI FUT per lot in ₹ -20026.0
Overall Winning Streak Profit in Supertrend Strategy for ^NSEI FUT per lot in ₹ 141930.0
Total Trades: 43
Count of Trades having Loss: 20
Count of Trades having Profit: 23
SuperTrend Strategy P&L ratio 1.15
Count of Trade percentage in Loss for ^NSEI in percent (POL) 47 %
Count of Trade percentage in Profit for ^NSEI in percent (POP) 53 %


In [45]:
df_sup.set_index('symbol', inplace=True)
df_sup

,entry_date,entry,vix_entry,exit_date,exit,vix_exit,position,diff,lot_size,pl,days_in_trade
symbol,,,,,,,,,,,
^NSEI,2016-03-10,7486.0,18.0,2016-09-29,8591.0,15.0,long buy,1105.0,50,55250.0,203.0
^NSEI,2016-09-29,8591.0,15.0,2017-01-05,8274.0,15.0,short sell,317.0,50,15873.0,98.0
^NSEI,2017-01-05,8274.0,15.0,2017-08-10,9820.0,12.0,long buy,1546.0,50,77300.0,217.0
^NSEI,2017-08-10,9820.0,12.0,2017-09-01,9974.0,14.0,short sell,-154.0,50,-7708.0,22.0
^NSEI,2017-09-01,9974.0,14.0,2017-09-25,9873.0,11.0,long buy,-102.0,50,-5100.0,24.0
^NSEI,2017-09-25,9873.0,11.0,2017-10-10,10017.0,13.0,short sell,-144.0,50,-7218.0,15.0
^NSEI,2017-10-10,10017.0,13.0,2017-11-14,10187.0,12.0,long buy,170.0,50,8500.0,35.0
^NSEI,2017-11-14,10187.0,12.0,2017-11-27,10400.0,13.0,short sell,-213.0,50,-10648.0,13.0
^NSEI,2017-11-27,10400.0,13.0,2017-12-01,10122.0,14.0,long buy,-278.0,50,-13900.0,4.0
